<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Semantic Clustering using Open Source Language Models in Database
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction:</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Semantic Clustering measures the degree to which two pieces of text relate in meaning, regardless of the exact wording. This captures relationships between words, sentences, or documents in ways that traditional keyword-based methods might miss.</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
            <li><strong>Information Retrieval:</strong> Improves search engines by retrieving documents or results that are semantically related to the user's query.</li>
            <li><strong>Text Classification:</strong> Categorizes text into predefined classes based on meaning, useful in spam detection, sentiment analysis, etc.</li>
            <li><strong>Question Answering:</strong> Matches questions to relevant answers by understanding their meaning.</li>
            <li><strong>Recommendation Systems:</strong> Suggests items (like products or content) based on similarity in user preferences or behaviors.</li>
            <li><strong>Plagiarism Detection:</strong> Identifies copied content even if paraphrased</li>
</ul>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
Teradata’s integration with LLMs and hosting capabilities on platforms like Amazon BedRock, along with the Open Analytics Framework, would enable customers to run NLP models at scale. The key challenges noted for on-prem customers—such as data movement latency and lack of access to cloud models—are valid. By bringing language models within Vantage, Teradata can provide a significant advantage to on-prem customers by allowing them to run NLP models without needing to move large amounts of data to and from external services.
</p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Downloading and installing additional software needed</b>

In [ ]:
%%capture
!pip install wordcloud nltk --quiet

In [ ]:
%%capture
!pip install --force-reinstall pillow --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>The above libraries have to be installed if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. Be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import time
import warnings
warnings.filterwarnings('ignore')
from teradataml import *
import getpass
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
display.max_rows = 5

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Initiate a connection to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Language_Model_SemanticClustering.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_cloud');"        # Takes 1 minute
%run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_local');"        # Takes 2 minutes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Confirmation for functions</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Before starting let us confirm that the required functions are installed.</p>

In [ ]:
from IPython.display import display, Markdown

df_check= DataFrame.from_query('''select count(*) as cnt from dbc.tablesV where databasename = 'ivsm';''')
if df_check.get_values()[0][0] >= 10:
    print('Functions are installed, please continue.')
else:
    print('Functions are not installed, please go to Instalization notebook before proceeding further')
    display(Markdown("[Initialization Notebook](./Initialization_and_Model_Load.ipynb)"))

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Create the Embeddings</b>

In [ ]:
tdf = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'consumer_complaints'))
tdf

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>3.1 Creation of the view with tokenized original texts</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This code creates a view named <code>v_complaints_tokenized_for_embeddings</code> that contains tokenized consumer complaint data for embedding purposes. It selects the <code>id</code>, <code>txt</code> (complaint text), <code>input_ids</code> (tokenized representations), and <code>attention_mask</code> from a tokenization function <code>ivsm.tokenizer_encode</code>.</b>

In [ ]:
execute_sql("""

replace view v_complaints_tokenized_for_embeddings as (
    select
        id,
        txt,
        IDS as input_ids,
        attention_mask
    from ivsm.tokenizer_encode(
        on (select top 1000 complaint_id as id, consumer_complaint_narrative as txt 
            from DEMO_ComplaintAnalysis.Consumer_Complaints)
        on (select model as tokenizer from embeddings_tokenizers where model_id = 'bge-small-en-v1.5') DIMENSION
        USING
            ColumnsToPreserve('id', 'txt')
            OutputFields('IDS', 'ATTENTION_MASK')
            MaxLength(1024)
            PadToMaxLength('True')
            TokenDataType('INT64')
    ) a
)
""")

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>3.2 Creation of the view with calculated binary embeddings</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This code creates a view named <code>complaints_embeddings</code> that stores the computed embeddings (vector representations) of consumer complaint texts. The embeddings are generated using the <code>ivsm.IVSM_score</code> function, which scores/tokenizes input data based on a specific model.</p>

In [ ]:
execute_sql("""

replace view complaints_embeddings as (
    select
            *
    from ivsm.IVSM_score(
            on v_complaints_tokenized_for_embeddings
            on (select * from embeddings_models where model_id = 'bge-small-en-v1.5') dimension
            using
                ColumnsToPreserve('id', 'txt')
                ModelType('ONNX')
                BinaryInputFields('input_ids', 'attention_mask')
                BinaryOutputFields('sentence_embedding')
                Caching('inquery')
        ) a
)

""")

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.3 Creating Final Embeddings table</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this last step we will create embeddings table creating a column for each embedding essentially converting an array to separate columns.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b> Do you want to generate the embeddings?</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Generating embeddings will take around <b>35-40 minutes.</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have already generated embeddings for the Consumer_Complaints and stored them in <b>Vantage</b> table.</p>
 
<center><img src="images/decision_emb_gen_cmt.svg" alt="embeddings_decision" width=300 height=400/></center>
 
<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><i><b>Note: If you would like to skip the embedding generation step to save the time and move quickly to next step, please enter "No" in the next prompt.</b></i></p>
</div>
 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>To save time, you can move to the already generated embeddings section. However, if you would like to see how we generate the embeddings, or if you need to generate the embeddings for a different dataset, then continue to the following section.</p>

In [ ]:
# Request user's input
generate = input("Do you want to generate embeddings? ('yes'/'no'): ")

# Check the user's input
if generate.lower() == 'yes':
    print("\nGreat! We'll start by generating embeddings.")

    print("\nGenerating embeddings and Saving to the database, please wait...")
    start = time.time()

    qry=("""

    create multiset table complaints_embeddings_store as (
        select
                *
        from ivsm.vector_to_columns(
                on complaints_embeddings
                using
                    ColumnsToPreserve('id', 'txt')
                    VectorDataType('FLOAT32')
                    VectorLength(384)
                    OutputColumnPrefix('emb_')
                    InputColumnName('sentence_embedding')
            ) a
    ) with data primary index(id)

    """)

    try:
        start = time.time()
        execute_sql(qry)
        end = time.time()
        print('Table Created')
        print("Total time to run tokenization+embeddings took = ",(end-start)/60, " min on 2nodes 4Amp VM")
        tdf_embeddings_store = DataFrame('complaints_embeddings_store')
        
        
    except:
        db_drop_table('complaints_embeddings_store')
        start = time.time()
        execute_sql(qry)
        end = time.time()
        print('Table Created')
        print("Total time to run tokenization+embeddings took = ",(end-start)/60, " min on 2nodes 4Amp VM")
        tdf_embeddings_store = DataFrame('complaints_embeddings_store')

    end = time.time()

    print(f"Total time to run tokenization+embeddings on {tdf_embeddings_store.shape[0]} rows = ", (end-start)/60, " min")

elif generate.lower() == 'no':
    tdf_embeddings_store = DataFrame('"DEMO_ComplaintAnalysis"."Complaints_Embeddings_Store"')
    print("\nLoaded embeddings from the Vantage table")

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Run K-Means on the Embeddings Store and then build final table with Cluster ID assignments to rows</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The K-means() function groups a set of observations into k clusters in which each observation belongs to the cluster with the nearest mean (cluster centers or cluster centroid). This algorithm minimizes the objective function, that is, the total Euclidean distance of all data points from the center of the cluster</p>

In [ ]:
embedding_column_list = [col for col in tdf_embeddings_store.columns if col not in ["id", "txt"]]

kmeans_out = KMeans(
    id_column="id",
    data=tdf_embeddings_store,
    target_columns=embedding_column_list,
    output_cluster_assignment=True,
    num_init=10,
    num_clusters=5,
    iter_max=50,
)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The output below shows cluster assignment for each row.</p>

In [ ]:
kmeans_out.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's check how many data points each cluster has.</p>

In [ ]:
kmeans_df = kmeans_out.result
d2 = kmeans_df.groupby('td_clusterid_kmeans').count()
d2

In [ ]:
clustered_df = tdf_embeddings_store.join(
    other = kmeans_df,
    on = ["id"],
    how = "inner",
    lprefix="l",
    rprefix="r"
)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's check a sample of data points from cluster number 2.</p>

In [ ]:
final_df = clustered_df[["l_id","txt","td_clusterid_kmeans"]]
final_df[final_df.td_clusterid_kmeans == 2]

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. WordCloud Visualization</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's visualize all the clusters through wordcloud visualization.</p>

In [ ]:
# Word Cloud all those clusters
for i in range(0, 5):
    filtered_df = final_df[final_df.td_clusterid_kmeans == i]
    df = filtered_df.to_pandas()
    total_rows = len(df)

    sw = ['x','xx','xxx','xxxx','xxxxx','xxxxxx']
    es = list(set(stopwords.words('english')))
    es.extend(sw)

    text_tokens = word_tokenize(' '.join(df['txt']),preserve_line=True)
    l_text_tokens = [item.lower() for item in text_tokens]
    tokens_without_sw = [word for word in l_text_tokens if word not in es]

    all_text = pd.Series(tokens_without_sw)

    vectorizer = TfidfVectorizer(ngram_range=(1,4))
    tfidf_matrix = vectorizer.fit_transform(all_text)

    col_sum = tfidf_matrix.sum(axis=0).A.squeeze()
    k = 5
    top_indices = np.argsort(col_sum)[-k:][::-1]

    dense = tfidf_matrix.todense()
    df = pd.DataFrame(dense, columns=vectorizer.get_feature_names_out())
    feature_names = vectorizer.get_feature_names_out()
    top_terms = feature_names[top_indices]
    cluster_name = '_'.join(top_terms).upper()

    print("Cluster #" + str(i)+": "+cluster_name+"\n # of Rows: "+str(total_rows)+"\n")
    # Generate a word cloud
    wordcloud = WordCloud(width=800, \
                          height=400, \
                          background_color='white',\
                          collocations=True,\
                          max_words=100,\
                          min_word_length=1, \
                         ).generate_from_frequencies(df.T.sum(axis=1))

    # Plot the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='gaussian')
    plt.axis('off')
    plt.title('Word Cloud for Text Data (TF-IDF)')
    plt.show()

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this demo we have seem that how we can run HuggingFace Embedding Model (BAAI/bge-small-1.5) in ONNX format and run it in database parallelly to create embeddings. We have done KMeans Clustering to group similar complaints.</p> 

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>6. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
tables = ['complaints_embeddings_store']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass

views = ['v_complaints_tokenized_for_embeddings','complaints_embeddings']

for view in views:
    try:
        db_drop_view(view_name=view)
    except:
        pass

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ComplaintAnalysis');"        # Takes 10 seconds

In [ ]:
remove_context()

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>